In [1]:
import os

import django
from django.core.management import call_command

# Simulate manage.py
os.environ["DJANGO_SETTINGS_MODULE"] = "arches_querysets.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

call_command("delete_test_data")
call_command("add_test_data")

Deleting test data...
Finished!
Creating test data...
Finished!


In [2]:
from arches_querysets.models import *

resource = ResourceTileTree.get_tiles(graph_slug="datatype_lookups").exclude(url=None).get()
print(resource.aliased_data)
print(resource.aliased_data.datatypes_1.aliased_data)

AliasedData(datatypes_n=[<TileTree: datatypes_n (66cf8a82-e565-419f-9a93-f59037a7b633)>], datatypes_1=<TileTree: datatypes_1 (89d65daf-ab75-4def-be95-7a8262231f61)>)
AliasedData(string={'en': {'value': 'forty-two', 'direction': 'ltr'}}, number=42, concept=<Value: Value object (d8c60bf4-e786-11e6-905a-b756ec83dad5)>, concept_list=[<Value: Value object (d8c60bf4-e786-11e6-905a-b756ec83dad5)>], date='2042-04-02', node_value='89d65daf-ab75-4def-be95-7a8262231f61', edtf=None, annotation=None, url={'url': 'http://www.42.com/', 'url_label': '42.com'}, resource_instance=<ResourceInstance: Datatype Lookups: Resource referencing 42 (e4e7f3a5-87a1-428d-8974-3ceccdbfe090)>, resource_instance_list=[<ResourceInstance: Datatype Lookups: Resource referencing 42 (e4e7f3a5-87a1-428d-8974-3ceccdbfe090)>], boolean=True, domain_value=None, domain_value_list=None, non_localized_string='forty-two', geojson_feature_collection=None, file_list=[{'url': 'http://www.archesproject.org/blog/static/42.png', 'name': 

In [3]:
resource = ResourceTileTree.get_tiles(
    graph_slug="datatype_lookups", as_representation=True
).exclude(url=None).get()

from pprint import pprint
pprint(resource.aliased_data)
pprint(resource.aliased_data.datatypes_1.aliased_data)

AliasedData(datatypes_n=[<TileTree: datatypes_n (66cf8a82-e565-419f-9a93-f59037a7b633)>],
            datatypes_1=<TileTree: datatypes_1 (89d65daf-ab75-4def-be95-7a8262231f61)>)
AliasedData(string={'display_value': 'forty-two',
                    'interchange_value': {'en': {'direction': 'ltr',
                                                 'value': 'forty-two'}}},
            number={'display_value': '42', 'interchange_value': 42},
            concept={'display_value': 'Arches',
                     'interchange_value': {'concept_id': '00000000-0000-0000-0000-000000000001',
                                           'language_id': 'en',
                                           'value': 'Arches',
                                           'valueid': 'd8c60bf4-e786-11e6-905a-b756ec83dad5',
                                           'valuetype_id': 'prefLabel'}},
            concept_list={'display_value': 'Arches',
                          'interchange_value': [{'concept_id': '0000

In [4]:
test_resources = ResourceTileTree.get_tiles("datatype_lookups", as_representation=True)
# This "shallow" lookup works whether or not the node is nested (todo: add test case)
resource = test_resources.filter(string__any_lang_contains="forty").first()
resource.aliased_data.datatypes_1.aliased_data.string

{'display_value': 'forty-two',
 'interchange_value': {'en': {'value': 'forty-two', 'direction': 'ltr'}}}

In [5]:
# You can save back with or without the display value / interchange_value wrapper
resource.aliased_data.datatypes_1.aliased_data.string = 'forty-three'
resource.save()
# But without a request, we assume anonymous user, and it ends up in provisional edits
resource.aliased_data.datatypes_1.provisionaledits

{'2': {'value': {'0cc0fd4f-6654-435e-84c1-44422190b5d0': 'forty-two',
   '3a3529cf-1ebc-4e8d-868a-d8bd21a37035': {'url': 'http://www.42.com/',
    'url_label': '42.com'},
   '585cbabb-7f5f-48a5-9848-8146b4bc7b7c': None,
   '5d3c9c04-3d13-4cec-9040-c1b0c0d7a824': True,
   '5ed5b53f-d5f8-4606-b526-eb5648b8092c': '2042-04-02T00:00:00.000-05:00',
   '6551184b-e402-4fa3-a7a8-2f9131e74d9d': [{'resourceId': 'e4e7f3a5-87a1-428d-8974-3ceccdbfe090',
     'ontologyProperty': '',
     'resourceXresourceId': 'f0c4b152-a7d1-4f4e-94c9-790d2b99e617',
     'inverseOntologyProperty': ''}],
   '6dac34ea-129c-4fbe-a27a-6feaf37ac5d5': None,
   '8028dc31-337a-43db-a1a8-4c2485b69ff8': [],
   '840e508c-024e-443f-9cfb-940588ab74e8': '89d65daf-ab75-4def-be95-7a8262231f61',
   '8db0bffa-f1af-408e-9d10-3191654665ab': None,
   '911755aa-f1c7-41e1-ba2f-bb4b7dcea59d': [{'url': '/files/6a52ba13-e4d6-4d03-9886-4c3615a930b8',
     'name': '42_accessibility_improvements.png',
     'type': 'image/png',
     'status': 'up

In [6]:
resource.aliased_data.datatypes_1.aliased_data.string = 'forty-three'
resource.save(force_admin=True)
resource.aliased_data.datatypes_1.aliased_data.string

{'display_value': 'forty-three',
 'interchange_value': {'en': {'value': 'forty-three', 'direction': 'ltr'}}}

In [7]:
# Custom lookups
resources = ResourceTileTree.get_tiles(
    "datatype_lookups", as_representation=True
).filter(string__any_lang_contains="forty")
for match in resources:
    print(match)

Datatype Lookups: Resource referencing 42 (e4e7f3a5-87a1-428d-8974-3ceccdbfe090)


In [8]:
resources = ResourceTileTree.get_tiles(
    "datatype_lookups", as_representation=True
).filter(resource_instance_list__contains=str(resource.pk))
for match in resources:
    print(match)

Datatype Lookups: Resource referencing 42 (e4e7f3a5-87a1-428d-8974-3ceccdbfe090)
